In [ ]:
import pandas as pd
import requests
import csv
import numpy as np
import re
from urllib.request import Request, urlopen
import ast
#spacy dependencies
import re 
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from sklearn.feature_extraction.text import CountVectorizer
from spacy.matcher import Matcher
from gensim import corpora
import pickle
import gensim
from sklearn.metrics.pairwise import cosine_similarity
#plotting tools
import matplotlib.pyplot as plt
%matplotlib inline
from statistics import mean, median, mode
import unicodedata
import pyLDAvis.gensim
from spacy import displacy
import scattertext as st
import inflect
import scattertext as st
from pprint import pprint
import explacy

In [ ]:
model_df = pd.read_csv('tokenized_model.csv')
model_df = model_df.where((pd.notnull(model_df)), None)
model_df.set_index('title',inplace=True)
model_df.drop_duplicates(subset='name',inplace=True)
model_plots = model_df['plot']

# Spacy Lemmatizer and tokenizer

### Tokenize text

In [ ]:
model_df.head()

In [ ]:
model_tokens = []

#tokenization the text and reinstantiated the function each time it is used to 
#prevent entity merging errors
for index,item in enumerate(model_plots):
    try:
        nlp = spacy.load('en_core_web_sm')
        stop_words = spacy.lang.en.stop_words.STOP_WORDS
        punctuations = string.punctuation

        #remove accendted characters
        def remove_accented_characters(text):
            cleantext = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            return cleantext

        matcher = Matcher(nlp.vocab)

        def spacy_tokenizer(row):

            cleantext = re.sub(r'((?<!\d)(\d{1,3}|\d{5,})(?!\d))|([^a-zA-Z0-9 -])','',row)
            cleantext2 = re.sub(r'(--)',' ', cleantext)
            #remove accented characters
            remove_accented_characters(cleantext2)

            # Creating our token object using spacy
            mytokens = nlp(cleantext2)

            #merge entity names into one token
            merge_nps = nlp.create_pipe("merge_entities")
            nlp.add_pipe(merge_nps)
            mytokens = nlp(cleantext2)

            # Lemmatizing each token and converting each token into lowercase
            mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

            # Removing stop words
            mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

            #remove all one letter words
            mytokens = [ word for word in mytokens if len(word)>1 ]
            
            #create single string of all tokens
            model_tokens.append(' '.join(mytokens))

            # return preprocessed list of tokens
            return mytokens

        spacy_tokenizer(item)

        print(f'{index} worked')
    except:
        model_tokens.append('Drop me!')
        print(f'{index} didnt work')
        pass


In [ ]:
#merge all columns into one string for vectorization
model_df['finished_data'] = model_df['score'].map(str) + ' ' + model_df['media_type'].map(str) + ' ' + model_df['Action'].map(str) + ' ' + model_df['Anime'].map(str) + ' ' + model_df['Children'].map(str) + ' ' + model_df['Comedy'].map(str) + ' ' + model_df['Cult'].map(str) + ' ' + model_df['Documentaries'].map(str) + ' ' + model_df['Dramas'].map(str) + ' ' + model_df['Fantasy'].map(str) + ' ' + model_df['Horror'].map(str) + ' ' + model_df['International'].map(str) + ' ' + model_df['Reality'].map(str) + ' ' + model_df['Sci-Fi'].map(str) + ' ' + model_df['Thrillers'].map(str) + ' ' + model_df['Romantic'].map(str) + ' ' + model_df['tokens']

# remove nullvalues from each entry
clean_finished_data = []
finished_data = model_df['finished_data']
for entry in finished_data:
    clean_entry = entry.replace('None ', '').lower()
    clean_finished_data.append(clean_entry)
    
model_df['finished_data'] = clean_finished_data

### Vectorize the data

In [ ]:
#this may be where specify to join proper nouns if there are two next to each other 
count = CountVectorizer(max_df = .7, min_df = 3)

In [ ]:
count_matrix = count.fit_transform(model_df['finished_data'])
pickle.dump(count_matrix, open('rec_matrix.pkl', 'wb'))

In [ ]:
rec_matrix = pickle.load(open('rec_matrix.pkl', 'rb'))

## Similarity Matrix

### Combine them into on gian matrix and just say if movie then recommend book and so on

In [ ]:
#creating a cosign similarity matrix
cosine_sim = cosine_similarity(rec_matrix, rec_matrix)

In [ ]:
pickle.dump(cosine_sim, open('rec_cosine_sim.pkl', 'wb'))

# Open the pickled matrix

In [ ]:
cosine_sim = pickle.load(open('rec_cosine_sim.pkl', 'rb'))

In [ ]:
#sample function to get the movie information along with its title
indices = pd.Series(model_df.index)
df_names = list(model_df['name'])

def recommendation(title, input_media_type,cosine_sim=cosine_sim):
    recommendations = []
    
    #getting the index of the movie that matches the title
    
    #If the desired output is a movie return a movie
    idx = indices[indices == title].index[0]

    #creating a series with the similarty scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending=False)
    
    top_10_indexes = list(score_series.iloc[1:1000].index)
        
    #wcreate a unique id for each entry and write a search function
    for item in top_10_indexes:
        recommendations.append(list(model_df.index)[item])
    
    #only get movies if a movie is asked for
    if input_media_type == 'movie':
        movie_recs = []
        for index1,rec in enumerate(recommendations):
            movie_title = rec
            for index2,name in enumerate(df_names):
                if name == movie_title:
                    movie_index = index2
                    searched_movie_title = model_df['name'][movie_index]
                    categorization = model_df['media_type'][movie_index]
                    if categorization != 'movie':
                        pass
                    else:
                        film_title = model_df['name'][movie_index]
                        film_score = model_df['score'][movie_index]
                        selected_image = model_df['image'][movie_index] 
                        selected_plot = model_df['plot'][movie_index]
                        movie_recs.append(searched_movie_title)
        return movie_recs[:5]
    
    #only get tv shows if a tv show is asked for
    if input_media_type == 'tv':
        tv_recs = []
        for index1,rec in enumerate(recommendations):
            tv_title = rec
            for index2,name in enumerate(df_names):
                if name == tv_title:
                    tv_index = index2
                    searched_tv_title = model_df['name'][tv_index]
                    categorization = model_df['media_type'][tv_index]
                    if categorization != 'tv':
                        pass
                    else:
                        tv_title = model_df['name'][tv_index]
                        tv_score = model_df['score'][tv_index]
                        selected_tv_image = model_df['image'][tv_index] 
                        selected_tv_plot = model_df['plot'][tv_index]
                        tv_recs.append(searched_tv_title)
        return tv_recs[:5]
    
    #only get books if a book is asked for
    if input_media_type == 'book':
        book_recs = []
        for index1,rec in enumerate(recommendations):
            book_title = rec
            for index2,name in enumerate(df_names):
                if name == book_title:
                    book_index = index2
                    searched_book_title = model_df['name'][book_index]
                    categorization = model_df['media_type'][book_index]
                    if categorization != 'book':
                        pass
                    else:
                        book_title = model_df['name'][book_index]
                        book_score = model_df['score'][book_index]
                        selected_book_image = model_df['image'][book_index] 
                        selected_book_plot = model_df['plot'][book_index]
                        book_recs.append(searched_book_title)
        return book_recs[:5]
    


In [ ]:
recommendation('Van Helsing', 'book')

# Visualizations

### Entity part of speech

In [ ]:
def visualize_entities_plot(title):
    try:
        names = model_df['name']
        plots = model_df['plot']
        for index,name in enumerate(names):
            if name == title:
                book_index = index
                plot_text = plots[book_index]
        
        #Visualize the entities
        nlp = spacy.load('en_core_web_sm')
        doc = nlp(plot_text)
        spacy.displacy.serve(doc, style='ent')
    except:
        print('That name is not in the directory!')

In [ ]:
visualize_entities_plot('The Da Vinci Code')

### Lemmitzation breakdown

In [ ]:
def visualize_lemmitization(title):
    try:
        names = model_df['name']
        plots = model_df['plot']
        for index,name in enumerate(names):
            if name == title:
                media_index = index
                plot_text = plots[media_index]
        
        #Visualize the entities
        nlp = spacy.load('en_core_web_sm')
        explacy.print_parse_info(nlp, plot_text)
    except:
        print('That name is not in the directory!')

In [ ]:
visualize_lemmitization('Narcos: Mexico')

In [ ]:
nlp = spacy.load('en_core_web_sm')
corpus = st.CorpusFromPandas(model_df,
                             category_col='media_type',
                             text_col='tokens',
                             nlp=nlp).build()

# Terms most associated with books

In [ ]:
print(list(corpus.get_scaled_f_scores_vs_background().index[:30]))

### Visualize the categories of the test

In [ ]:
feat_builder = st.FeatsFromOnlyEmpath()

In [ ]:
empath_corpus = st.CorpusFromParsedDocuments(model_df,
                                             category_col='media_type',
                                             feats_from_spacy_doc=feat_builder,
                                             parsed_col='tokens').build()

In [ ]:
html = st.produce_scattertext_explorer(empath_corpus,
                                       category='book',
                                       category_name='Books',
                                       not_category_name='Netflix',
                                       width_in_pixels=1000,
                                       metadata=model_df['media_type'],
                                       use_non_text_features=True,
                                       use_full_doc=True,
                                       topic_model_term_lists=feat_builder.get_top_model_term_lists())

In [ ]:
open("book-v-netflix-empathy.html", 'wb').write(html.encode('utf-8'))